In [15]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [16]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [17]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [18]:
from pyspark.sql import SparkSession

In [19]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [20]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

**2. Data Understanding using SparkSQL**

In [21]:
! wget https://storage.googleapis.com/class25jan2022/share/2008.csv

--2022-02-03 15:57:38--  https://storage.googleapis.com/class25jan2022/share/2008.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.146.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.146.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [text/csv]
Saving to: ‘2008.csv.1’

2008.csv.1          100%[===================>] 657.48M   184MB/s    in 3.8s    

2022-02-03 15:57:42 (175 MB/s) - ‘2008.csv.1’ saved [689413344/689413344]



In [22]:
! wc -l ./2008.csv

7009729 ./2008.csv


In [23]:
! head -3 2008.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA


In [24]:
raw_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load('2008.csv')

In [25]:
raw_df.count()

7009728

In [26]:
raw_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [27]:
raw_df\
.select(['DepTime','TaxiOut','TaxiIn',\
                     'DayOfWeek','Distance','ArrDelay'])\
                     .describe().show()

+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|summary|           DepTime|           TaxiOut|           TaxiIn|         DayOfWeek|         Distance|         ArrDelay|
+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|  count|           7009728|           7009728|          7009728|           7009728|          7009728|          7009728|
|   mean|1333.8300461105448|16.453045177492882|6.860851704974527|3.9241815088973495|726.3870294253928| 8.16845238729114|
| stddev|478.06889486629836|11.332798654232155|4.933649371300466|1.9882589459851212|562.1018034840403|38.50193694882867|
|    min|                 1|                 0|                0|                 1|              100|               -1|
|    max|                NA|                NA|               NA|                 7|              999|               NA|
+-------+------------------+----

In [28]:
from pyspark.sql.functions import *

In [29]:
raw_df.select(col("DepTime").cast('Integer')).show()

+-------+
|DepTime|
+-------+
|   2003|
|    754|
|    628|
|    926|
|   1829|
|   1940|
|   1937|
|   1039|
|    617|
|   1620|
|    706|
|   1644|
|   1426|
|    715|
|   1702|
|   1029|
|   1452|
|    754|
|   1323|
|   1416|
+-------+
only showing top 20 rows



In [30]:
raw_df_test = raw_df.filter(raw_df.DepTime=='12')

In [31]:
raw_df_test.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|        27|        7|     12|      2130|    110|      2235|           WN|     2721

In [32]:
raw_df_test.select(col("DepTime").cast('Integer')).show()

+-------+
|DepTime|
+-------+
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
|     12|
+-------+
only showing top 20 rows



In [33]:
raw_df.filter(raw_df.DepTime=='24').show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         7|        1|     24|      2055|    312|      2340|           YV|     7248

In [34]:
raw_df.select('DepTime').orderBy('DepTime',ascending=True).show()

+-------+
|DepTime|
+-------+
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
+-------+
only showing top 20 rows



In [35]:
from pyspark.sql.functions import *

In [36]:
def DepTimePeriod(x):
  if x is None:
    time_grp = 'null'
  elif (x>=1 and x<6)or (x>=100 and x<=559):
    time_grp = '00.01-05.59'
  elif (x>=6 and x<12) or (x>=600 and x<=1159):
    time_grp = '06.00-11.59'
  elif (x>=12 and x<18) or (x>=1200 and x<=1759):
    time_grp = '12.00-17.59'
  elif (x>=18 and x<=24) or (x>=1800 and x<=2400):
    time_grp = '18.00-24.00'
  else:
    time_grp ='null'

  return time_grp

In [37]:
timeperiod = udf(lambda x:DepTimePeriod(x) )

In [38]:
depTimePeriod = raw_df.withColumn('DepTime',timeperiod(col('DepTime').cast("Integer")))\
.groupBy('DepTime').count().orderBy('DepTime')

In [39]:
depTimePeriod.show()

+-----------+-------+
|    DepTime|  count|
+-----------+-------+
|00.01-05.59| 162405|
|06.00-11.59|2645956|
|12.00-17.59|2556733|
|18.00-24.00|1497435|
|       null| 147199|
+-----------+-------+



In [40]:
# depTimePeriod = raw_df.withColumn('DepTime',timeperiod(col('DepTime').cast("Integer")))
# depTimePeriod.show()

In [41]:
raw_df.groupBy('month').count().orderBy('count',ascending=False).show()

+-----+------+
|month| count|
+-----+------+
|    7|627931|
|    3|616090|
|    8|612279|
|    6|608665|
|    5|606293|
|    1|605765|
|    4|598126|
|    2|569236|
|   10|556205|
|   12|544958|
|    9|540908|
|   11|523272|
+-----+------+



In [42]:
from pyspark.sql import functions as sparkf

In [43]:
raw_df.groupBy('month').agg({'month':'count','ArrDelay':'avg'})\
.withColumnRenamed('count(month)','numMonth')\
.withColumnRenamed('avg(ArrDelay)','avgArrDelay')\
.orderBy(sparkf.col('numMonth').desc(),sparkf.col('avgArrDelay').desc()).show()

+-----+--------+------------------+
|month|numMonth|       avgArrDelay|
+-----+--------+------------------+
|    7|  627931| 9.975049681276131|
|    3|  616090| 11.19236458018227|
|    8|  612279|  6.91091468997087|
|    6|  608665|13.266756009659792|
|    5|  606293| 5.978448290248828|
|    1|  605765|10.188855960349496|
|    4|  598126| 6.807297481094145|
|    2|  569236|13.077836997760205|
|   10|  556205|0.4154954706912698|
|   12|  544958|16.680505081496417|
|    9|  540908|0.6977328787273043|
|   11|  523272| 2.015857969430839|
+-----+--------+------------------+

